# Predicting Pitches
For our project, we'll aim to solve the problem of predicting the next pitch a Major League Baseball Pitcher will throw during an at bat

In [1]:
import utils.utils as utils
import pandas as pd

## Part 1: Preprocessing the data

In [8]:
# get the data
pitch_data = utils.get_pitch_data()
pitch_data.head()

,pitch_data_id,season,pitcher_id,team_id,team_abbrev,era,wins,losses,throws,b1_id,...,is_runner_on_second,is_runner_on_third,runs_pitcher_team,runs_batter_team,game_id,inning_id,half_inning_id,at_bat_id,gid,p1_pitch_type
0,282234,2018,598264,111,BOS,2.6,2.0,2.0,R,607208,...,False,False,0,0,06c8d771-e316-471b-a3ac-bbed7454b237,a105f7ad-9d9a-422f-9050-288dc94f1ffc,4ed07502-0d97-45ef-8572-ed494307f2a7,42a982f7-8602-444f-8f69-d2db01059d09,gid_2018_07_02_bosmlb_wasmlb_1,CU
1,282235,2018,598264,111,BOS,2.6,2.0,2.0,R,607208,...,False,False,0,0,06c8d771-e316-471b-a3ac-bbed7454b237,a105f7ad-9d9a-422f-9050-288dc94f1ffc,4ed07502-0d97-45ef-8572-ed494307f2a7,42a982f7-8602-444f-8f69-d2db01059d09,gid_2018_07_02_bosmlb_wasmlb_1,CU
2,282236,2018,598264,111,BOS,2.6,2.0,2.0,R,607208,...,False,False,0,0,06c8d771-e316-471b-a3ac-bbed7454b237,a105f7ad-9d9a-422f-9050-288dc94f1ffc,4ed07502-0d97-45ef-8572-ed494307f2a7,42a982f7-8602-444f-8f69-d2db01059d09,gid_2018_07_02_bosmlb_wasmlb_1,FF
3,282237,2018,598264,111,BOS,2.6,2.0,2.0,R,607208,...,False,False,0,0,06c8d771-e316-471b-a3ac-bbed7454b237,a105f7ad-9d9a-422f-9050-288dc94f1ffc,4ed07502-0d97-45ef-8572-ed494307f2a7,42a982f7-8602-444f-8f69-d2db01059d09,gid_2018_07_02_bosmlb_wasmlb_1,FF
4,282238,2018,598264,111,BOS,2.6,2.0,2.0,R,607208,...,False,False,0,0,06c8d771-e316-471b-a3ac-bbed7454b237,a105f7ad-9d9a-422f-9050-288dc94f1ffc,4ed07502-0d97-45ef-8572-ed494307f2a7,42a982f7-8602-444f-8f69-d2db01059d09,gid_2018_07_02_bosmlb_wasmlb_1,CU


In [9]:
pitch_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 72 columns):
pitch_data_id          10000 non-null int64
season                 10000 non-null int64
pitcher_id             10000 non-null int64
team_id                10000 non-null object
team_abbrev            10000 non-null object
era                    10000 non-null float64
wins                   9991 non-null float64
losses                 9991 non-null float64
throws                 10000 non-null object
b1_id                  10000 non-null int64
b1_team_id             10000 non-null object
b1_stand               10000 non-null object
b1_height              10000 non-null int64
b1_bats                10000 non-null object
b1_avg                 10000 non-null float64
b1_hr                  10000 non-null int64
b1_rbi                 10000 non-null int64
b1_bat_order           9347 non-null float64
b1_game_position       9347 non-null object
p1_pitch_id            10000 non-null objec

In [10]:
cols_to_drop=['p1_pitch_id','p0_pitch_id','result_type','pitch_data_id','season','team_id','game_id','inning_id','half_inning_id','at_bat_id','gid']
# drop unneeded cols
pitch_data = utils.drop_columns_by_list(pitch_data,cols_to_drop)
# drop pickoff attempts
pitch_data = utils.drop_pickoffs(pitch_data)
# set category of cols
pitch_data = utils.set_dtypes(pitch_data)
# drop nans
pitch_data = pitch_data.dropna()
all_data = pitch_data

In [5]:
pitch_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8872 entries, 0 to 9999
Data columns (total 61 columns):
pitcher_id             8872 non-null object
team_abbrev            8872 non-null object
era                    8872 non-null float64
wins                   8872 non-null float64
losses                 8872 non-null float64
throws                 8872 non-null object
b1_id                  8872 non-null object
b1_team_id             8872 non-null object
b1_stand               8872 non-null int64
b1_height              8872 non-null int64
b1_bats                8872 non-null object
b1_avg                 8872 non-null float64
b1_hr                  8872 non-null int64
b1_rbi                 8872 non-null int64
b1_bat_order           8872 non-null float64
b1_game_position       8872 non-null object
p1_pitch_seqno         8872 non-null int64
p0_pitch_seqno         8872 non-null float64
p0_inning              8872 non-null float64
result_type_simple     8872 non-null object
x          

In [7]:
pitch_types = pitch_data.loc[:,'p1_pitch_type']
Y = utils.encode_pitch_types(pitch_types)
X = pitch_data.drop('p1_pitch_type',axis=1)
# one hot encode necessary cols
X = utils.one_hot_encode(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8743 entries, 0 to 9999
Columns: 158 entries, era to inning_16
dtypes: float64(35), int64(15), uint8(108)
memory usage: 4.3 MB


In [7]:
X_train, X_test, y_train, y_test = utils.split_dataset_into_train_and_test(X,Y)

In [8]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7166 entries, 6012 to 8143
Columns: 1148 entries, era to inning_14
dtypes: float64(35), int64(15), uint8(1098)
memory usage: 10.3 MB


## Part2: Train an XGBoost multiclassifier

In [7]:
import models.xgboost_model as xgb_model

In [8]:
xgb_multi_class_model = xgb_model.get_multi_class_classifier_model()

In [11]:
xgb_model.fit_multi_class_model(model=xgb_multi_class_model,x_train=X_train,y_train=y_train,x_test=X_test,y_test=y_test,save_location='./training/xgb_multi_class.bin')


Model Report
Accuracy (Train) : 1.0
Accuracy (Test) : 0.4447544642857143


It looks like we're overfitting. I'll remove the pitcher and batter ids

In [5]:
pitch_data = all_data
# remove the pitch
cols_to_drop=['pitcher_id','b1_id']
# drop unneeded cols
pitch_data = utils.drop_columns_by_list(pitch_data,cols_to_drop)
X = pitch_data.drop('p1_pitch_type',axis=1)
# one hot encode necessary cols
X = utils.one_hot_encode(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8743 entries, 0 to 9999
Columns: 158 entries, era to inning_16
dtypes: float64(35), int64(15), uint8(108)
memory usage: 4.3 MB


In [6]:
X_train, X_test, y_train, y_test = utils.split_dataset_into_train_and_test(X,Y)

NameError: name 'Y' is not defined

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7097 entries, 7039 to 8201
Columns: 155 entries, era to inning_15
dtypes: float64(35), int64(15), uint8(105)
memory usage: 3.5 MB


In [12]:
xgb_multi_class_model = xgb_model.get_multi_class_classifier_model()

In [14]:
xgb_model.fit_multi_class_model(model=xgb_multi_class_model,x_train=X_train,y_train=y_train,x_test=X_test,y_test=y_test,save_location='./training/xgb_multi_class_no_pitcher_batter_id.bin',useTrainCV=True)

/home/jpalomares/.local/lib/python3.6/site-packages/xgboost/core.py:587: FutureWarning: Series.base is deprecated and will be removed in a future version
  if getattr(data, 'base', None) is not None and \


NameError: name 'alg' is not defined